In [1]:
from datetime import datetime
from datasets import load_dataset
from rich.pretty import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from transformers import AutoTokenizer, AutoConfig
from transformers.models.bert.modeling_bert import BertPreTrainedModel, BertModel
from torch.utils.data import Dataset, DataLoader
import os
import torch.nn as nn
import copy
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.nn.functional as Fun

c:\Users\simsIab\anaconda3\envs\bertClassfication\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if os.path.exists('data') == False:
    os.makedirs('data')

if os.path.exists('model') == False:
    os.makedirs('model')

if os.path.exists('result_img') == False:
    os.makedirs('result_img')


parameters = {
    "num_class": 4,
    "time": str(datetime.now().strftime("%m-%d-%H-%M")),
    "seed": 42,
    "train_size": 3000,
    "val_size": 500,
    # Hyperparameters
    "model_name": 'BERT', # If U have a lot of different models, it is easy for U to know what it is
    "config": 'nbroad/ESG-BERT', # which pre-trained model config U use
    "learning_rate": 1e-4, # the speed that model learn
    "epochs": 3, # If U would fine-tune it, the epochs didn't need to set too much
    "max_len": 512, # the max length of input tokens in the BERT model
    "batch_size": 64, 
    "dropout": 0.1, # how random amount will be give up
    "activation": 'tanh',
    "hidden_dim": 384,
}

In [3]:
# 載入資料集
dataset = load_dataset("FinanceMTEB/ESG")

In [4]:
pprint(dataset)
pprint(dataset['train'][0])

DatasetDict({
│   train: Dataset({
│   │   features: ['text', 'label', 'label_text'],
│   │   num_rows: 3000
│   })
│   test: Dataset({
│   │   features: ['text', 'label', 'label_text'],
│   │   num_rows: 1000
│   })
})

{
│   'text': 'This explains the increase in high-risk events in 2017, and the subsequent improvement in 2018 and 2019, as the action items implemented supported the decreasing number of high-risk incidents.',
│   'label': 3,
│   'label_text': 'social'
}

In [5]:
# 將資料集轉為 DataFrame 的格式

train_data = []
test_data = []

for data in dataset['train']:
  train_data.append({'text':data['text'], 'label':data['label']})
for data in dataset['test']:
  test_data.append({'text':data['text'], 'label':data['label']})

train_df = pd.DataFrame(train_data, columns=['text', 'label'])
test_df = pd.DataFrame(test_data, columns=['text', 'label'])
train_df.head(5)

,text,label
0,This explains the increase in high-risk events...,3
1,Focusing on shared value creation in 2019 allo...,3
2,We included information on our policy and lobb...,1
3,"As of December 31, 2011, the project was appro...",2
4,Independent lines of evidence continue to indi...,0


In [6]:
# 檢查各種 label 比例是否有資料不平衡問題

train_df.label.value_counts() / len(train_df)

label
2    0.501000
3    0.267333
0    0.203000
1    0.028667
Name: count, dtype: float64

In [7]:
# random_state 是固定資料 random 的結果，才不會每次切出來的資料集不一樣
# train_size：指定 output 中前者資料數量占比 
val_df, test_df = train_test_split(test_df, random_state=parameters['seed'], train_size=0.5)
print('# of train_df:', len(train_df))
print('# of val_df:', len(val_df))
print('# of test_df data:', len(test_df))

# save data
# 這裡指定sep='\t'，且不儲存DataFrame前面的index
train_df.to_csv('./data/train.tsv', sep='\t', index=False)
val_df.to_csv('./data/val.tsv', sep='\t', index=False)
test_df.to_csv('./data/test.tsv', sep='\t', index=False)

# of train_df: 3000
# of val_df: 500
# of test_df data: 500


In [8]:
# 如果你發現在使用時常常有一長串的 warning 跳出來，可以用這行指令把它關掉
# transformers.logging.set_verbosity_error() # Close the warning message

config_name = 'nbroad/ESG-BERT' # 使用 ESG-BERT
# .from_pretrained() 就是用現有的模型繼續做
tokenizer = AutoTokenizer.from_pretrained(config_name)

In [9]:
# 檢查 tokenizer 的 max_length
max_length = tokenizer.model_max_length
max_length

512

In [10]:
# 檢查 model 的 hidden_size
config = AutoConfig.from_pretrained(config_name)
config.hidden_size

768

In [11]:
torch.tensor([[0, 0, 1, 0],[0, 0, 1, 0],[0, 0, 1, 0]]).size()

torch.Size([3, 4])

In [12]:
# 製作 Dataset

class CustomDataset(Dataset):
    def __init__(self, mode, df, specify, args):
        assert mode in ["train", "val", "test"]
        self.mode = mode                                                    # 指定為 "train", "val", "test"
        self.df = df                                                        # 指定資料集
        self.specify = specify                                              # 要進行 tokenize 的欄位名稱
        if self.mode != 'test':
          self.label = df['label']                                          # 當資料為 "train"、"val" 時，進行 label
        self.tokenizer = AutoTokenizer.from_pretrained(args["config"])      # 指定 tokenize 的方法
        self.max_len = args["max_len"]                                      # 指定 tokenize 的最大長度
        self.num_class = args["num_class"]                                  # 指定 label 有幾種類別
        
    # 回傳 Dataset 的資料筆數
    def __len__(self):
        return len(self.df)

    # 當 self.num_class 大於2時，對 label 進行 one hot encodding
    # 例如 label 為 2 ， num_classes = 4 時
    # 則回傳 torch.tensor([0, 0, 1, 0]) ，其維度為 torch.size([num_classes])
    def one_hot_label(self, label):
        return Fun.one_hot(torch.tensor(label), num_classes = self.num_class)
    
    # 進行 tokenize
    def tokenize(self, input_text):
        inputs = self.tokenizer.encode_plus(
            input_text,                     # 指定文本
            max_length = self.max_len,      # 指定最長文本長度
            truncation = True,              # 開啟截斷功能
            padding = 'max_length'          # 依照 max_length 進行 padding
        )
        ids = inputs['input_ids']                 # size(self.max_len)
        mask = inputs['attention_mask']           # size(self.max_len)
        token_type_ids = inputs["token_type_ids"] # size(self.max_len)
        
        return ids, mask, token_type_ids

    # 獲得單一一筆資料
    def __getitem__(self, index):
        sentence = str(self.df[self.specify][index])            # 取出單筆資料的字串
        ids, mask, token_type_ids = self.tokenize(sentence)     # 進行 tokenize

        if self.mode == "test":
            # 回傳 input_ids, attention_mask, totken_type_ids
            # 需回傳 tensor 型態，其每個維度為 torch.Size([self.max_len])
            return torch.tensor(ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), \
                torch.tensor(token_type_ids, dtype=torch.long)
        else:
            if self.num_class > 2:     # 如 self.num_class > 2 時，進行 one hot encodding
            # 回傳 input_ids, attention_mask, totken_type_ids, labels
            # 需回傳 tensor 型態，其前三者維度為 torch.Size([self.max_len])，最後者維度為 torch.Size([self.num_class])
              return torch.tensor(ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), \
                torch.tensor(token_type_ids, dtype=torch.long), self.one_hot_label(self.label[index])
            else:
            # 回傳 input_ids, attention_mask, totken_type_ids, labels
            # 需回傳 tensor 型態，其前三者維度為 torch.Size([self.max_len])，最後者維度為 torch.Size([1])
              return torch.tensor(ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), \
                torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(self.label[index], dtype=torch.long)

In [13]:
# load training data
# 可以先 sample 部分資料去跑模型，有助於快速調整模型架構，畢竟資料愈多跑愈久
# 將 Dataset 放入 DataLoader 中，並指定 batch_size
train_df = pd.read_csv('./data/train.tsv', sep = '\t').sample(parameters['train_size'], random_state=parameters['seed']).reset_index(drop=True)
train_dataset = CustomDataset('train', train_df, 'text', parameters)
train_loader = DataLoader(train_dataset, batch_size=parameters['batch_size'], shuffle=True)

# load validation data
val_df = pd.read_csv('./data/val.tsv', sep = '\t').sample(parameters['val_size'], random_state=parameters['seed']).reset_index(drop=True)
val_dataset = CustomDataset('val', val_df, 'text', parameters)
val_loader = DataLoader(val_dataset, batch_size=parameters['batch_size'], shuffle=True)

In [14]:
for data in val_loader:
    # pprint(data)
    # print(len(data))
    ids, masks, token_type_ids, labels = data
    print(ids.shape)
    print(masks.shape)
    print(token_type_ids.shape)
    print(labels.shape)
    break

torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 512])
torch.Size([64, 4])


In [15]:
# 定義激活函數
def get_activation(activation):
    if activation == 'Prelu':
        return nn.PReLU()
    elif activation == 'relu':
        return nn.ReLU()
    elif activation == 'sigmoid':
        return nn.Sigmoid()
    elif activation == 'gelu':
        return nn.GELU()
    elif activation == 'LeakyReLU':
        return nn.LeakyReLU()
    else:
        return nn.Tanh()

# Dense Layer
# It is composed of linear, dropout, and activation layers.
class Dense(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate, activation='tanh'):
        super(Dense, self).__init__()
        self.hidden_layer = nn.Linear(input_dim, output_dim)    # 全連接層，輸入維度是 input_dim，輸出維度是 output_dim
        self.dropout = nn.Dropout(dropout_rate)                 # 定義 Dropout 層
        self.activation = get_activation(activation)            # 指定激活函數
        nn.init.xavier_uniform_(self.hidden_layer.weight)       # Xavier 初始化，有助於模型收斂
    def forward(self, inputs):
        logits = self.hidden_layer(inputs)
        logits = self.dropout(logits)
        logits = self.activation(logits)
        return logits

# multi-layers
def _get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

# Hidden Layers
# It means there are many dense layers with the same dimension
class HiddenLayers(nn.Module):
    def __init__(self, dense_layer, num_layers):
        super(HiddenLayers, self).__init__()
        self.hidden_layers = _get_clones(dense_layer, num_layers)   # dense_layer 為 Dense 物件，num_layers 為複製幾個 Dense 物件
    def forward(self, output):
        for layer in self.hidden_layers:
            output = layer(output)
        return output

In [16]:
# BERT Model
class BertClassifier(BertPreTrainedModel):
    def __init__(self, config, args):
        '''
        當呼叫 BertClassifier.from_pretrained(parameters['config'], parameters) 時會執行
        config: pretrained Model 的參數， config = AutoConfig.from_pretrained(config_name)
        args: 自行添加的參數，會將 parameters 傳入
        '''
        super(BertClassifier, self).__init__(config)
        self.bert = BertModel(config)               # 初始化 Bert 模型
        self.num_labels = args["num_class"]         # 指定 label 種類數量
        self.dense = Dense(config.hidden_size, args["hidden_dim"], args["dropout"], args["activation"])     # 宣告 Dense 作為 fully connected
        self.classifier = Dense(args["hidden_dim"], self.num_labels, args["dropout"], args["activation"])   # 宣告 Dense 作為 linear classfier
        self.init_weights()                         # 初始化權重
    
    # forward function, data in the model will do this
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None,
                head_mask=None, inputs_embeds=None, labels=None, output_attentions=None,
                output_hidden_states=None, return_dict=None):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        # 使用原始 BertModel 進行預測
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict
        )

        # 原始 BERT 模型的回傳:
        # outputs.keys() -> odict_keys(['last_hidden_state', 'pooler_output'])
        # outputs.last_hidden_state.shape -> torch.Size([batch_size, args["max_length"], config.hidden_size])
        # outputs.pooler_output.shape -> torch.Size([batch_size, config.hidden_size])

        # pooler_output 維度為 [batch_size, config.hidden_size] ，其實就是 last_hidden_state 的第一個 token ， 即 [CLS] logits
        pooled_output = outputs[1]                  # (batch_size, config.hidden_size)

        # 加上 Dense ，即將上游任務 Embedding 結果放到下游任務中
        pooled_output = self.dense(pooled_output)   # (batch_size, args["hidden_dim"]) 添加一層 NN ，作為正式進去線性分類層的緩衝(先降維度)
        logits = self.classifier(pooled_output)     # (batch_size, self.num_labels) 線性分類層
        return logits

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   # 確認GPU可否使用
model = BertClassifier.from_pretrained(parameters['config'], parameters).to(device) # 載入域訓練模型與傳入參數，並放到GPU計算
loss_fct = nn.CrossEntropyLoss()    # 使用 cross entrophy loss

## You can custom your optimizer (e.g. SGD .etc) ##
# we use Adam here
optimizer = torch.optim.Adam(model.parameters(), lr=parameters['learning_rate'], betas=(0.9, 0.999), eps=1e-9)

## You also can add your custom scheduler ##
# num_train_steps = len(train_loader) * parameters['epochs]
# scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1 * num_train_steps), num_training_steps=num_train_steps, num_cycles=1)

Some weights of BertClassifier were not initialized from the model checkpoint at nbroad/ESG-BERT and are newly initialized: ['classifier.hidden_layer.bias', 'classifier.hidden_layer.weight', 'dense.hidden_layer.bias', 'dense.hidden_layer.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# save model to path
def save_checkpoint(save_path, model):
    if save_path == None:
        return
    torch.save(model.state_dict(), save_path)
    print(f'Model saved to ==> {save_path}')

# load model from path
def load_checkpoint(load_path, model, device):
    if load_path==None:
        return
    state_dict = torch.load(load_path, map_location=device)
    print(f'\nModel loaded from <== {load_path}')

    model.load_state_dict(state_dict)
    return model

In [19]:
def get_class(logits):
    '''
    舉例說明:
    輸入為: logits = torch.tensor([
      [2.5, 0.3, 1],  # 第 1 個樣本
      [1.0, 3.1, 5],  # 第 2 個樣本
      [0.2, 0.9, 2.1],  # 第 3 個樣本
      [1.5, 1.5, 0.5]   # 第 4 個樣本
    ])
    輸出為: tensor([0, 2, 2, 0])，維度為 torch.size([batch_size])
    '''
    y_pred = torch.argmax(logits, dim = 1)      # 從 logits 第一維找出最大值的位置
    return y_pred

# calculate confusion metrics
def cal_metrics(pred, ans, method):
    '''
    Parameter
    ---------
    pred: [list], predict class
    ans: [list], true class
    method: 'micro', 'weighted', 'macro'. # 如果有多分類的話計算上會有差別
    'micro'：基於全體樣本計算，計算所有樣本的總體效果。
    'macro'：對每個類別分別計算指標，然後取平均，不考慮類別樣本數差異。
    'weighted'：對每個類別分別計算指標，然後根據樣本數取加權平均。
    ---------
    '''
    # 將 tensor 移動到 CPU 並轉成 numpy
    if pred.get_device() != 'cpu':
        pred = pred.detach().cpu().numpy()
    if ans.get_device() != 'cpu':
        ans = ans.detach().cpu().numpy()
    # sklearn.metrics 的各式計算方法須將 pred ans 兩個 label list 放入 
    # 將 zero_division 設為 0，表示當所有預測皆錯誤時，將結果視為 0 
    rec = recall_score(pred, ans, average=method, zero_division=0)
    f1 = f1_score(pred, ans, average=method, zero_division=0)
    prec = precision_score(pred, ans, average=method, zero_division=0)
    acc = accuracy_score(pred, ans)
    return acc, f1, rec, prec

In [20]:
# evaluate dataloader
def evaluate(model, data_loader, device):
    val_loss, val_acc, val_f1, val_rec, val_prec = 0.0, 0.0, 0.0, 0.0, 0.0
    step_count = 0
    loss_fct = nn.CrossEntropyLoss()
    model.eval()
    with torch.no_grad():
        for data in data_loader:
            ids, masks, token_type_ids, labels = [t.to(device) for t in data]

            logits = model(input_ids = ids,
                    token_type_ids = token_type_ids,
                    attention_mask = masks)
            
            acc, f1, rec, prec = cal_metrics(get_class(logits), get_class(labels), 'weighted')
            loss = loss_fct(logits, get_class(labels))

            val_loss += loss.item()
            val_acc += acc
            val_f1 += f1
            val_rec += rec
            val_prec += prec
            step_count+=1

        val_loss = val_loss / step_count
        val_acc = val_acc / step_count
        val_f1 = val_f1 / step_count
        val_rec = val_rec / step_count
        val_prec = val_prec / step_count

    return val_loss, val_acc, val_f1, val_rec, val_prec

In [21]:
def train(model, train_loader, val_loader, optimizer, args, device):
    time_stamp = str(datetime.now().strftime("%m-%d-%H-%M"))
    metrics = ['loss', 'acc', 'f1', 'rec', 'prec']
    mode = ['train_', 'val_']
    record = {s+m :[] for s in mode for m in metrics}

    loss_fct = nn.CrossEntropyLoss()

    for epoch in range(args["epochs"]):

        st_time = time.time()
        train_loss, train_acc, train_f1, train_rec, train_prec = 0.0, 0.0, 0.0, 0.0, 0.0
        step_count = 0

        model.train()
        for data in train_loader:

            # labels 維度為 torch.size([batch_size, args[num_class]])
            ids, masks, token_type_ids, labels = [t.to(device) for t in data]

            optimizer.zero_grad()
    
            # logits 維度為 torch.size([batch_size, args[num_class]])
            logits = model(input_ids = ids,
                    token_type_ids = token_type_ids,
                    attention_mask = masks)

            acc, f1, rec, prec = cal_metrics(get_class(logits), get_class(labels), 'weighted')

            # CrossEntropy 計算
            # pred 必須是 logits， 會自動轉成 softmax
            # labels 必須是類別
            loss = loss_fct(logits, get_class(labels))

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_acc += acc
            train_f1 += f1
            train_rec += rec
            train_prec += prec
            step_count += 1

        val_loss, val_acc, val_f1, val_rec, val_prec = evaluate(model, val_loader, device)

        train_loss = train_loss / step_count
        train_acc = train_acc / step_count
        train_f1 = train_f1 / step_count
        train_rec = train_rec / step_count
        train_prec = train_prec / step_count

        print('[epoch %d] cost time: %.4f s'%(epoch + 1, time.time() - st_time))
        print('         loss     acc     f1      rec    prec')
        print('train | %.4f, %.4f, %.4f, %.4f, %.4f'%(train_loss, train_acc, train_f1, train_rec, train_prec))
        print('val  | %.4f, %.4f, %.4f, %.4f, %.4f\n'%(val_loss, val_acc, val_f1, val_rec, val_prec))

        # record training metrics of each training epoch
        record['train_loss'].append(train_loss)
        record['train_acc'].append(train_acc)
        record['train_f1'].append(train_f1)
        record['train_rec'].append(train_rec)
        record['train_prec'].append(train_prec)
    
        record['val_loss'].append(val_loss)
        record['val_acc'].append(val_acc)
        record['val_f1'].append(val_f1)
        record['val_rec'].append(val_rec)
        record['val_prec'].append(val_prec)

        # save model
        model_filename = f"./model/{time_stamp}_epoch{epoch}.pt"
        save_checkpoint(model_filename, model)

    return record

In [22]:
import matplotlib.pyplot as plt

# draw the learning curve
def draw_pic(record, name, img_save=False, show=False):
    x_ticks = range(1, parameters['epochs']+1)

    plt.figure(figsize=(6, 3))

    plt.plot(x_ticks, record['train_'+name], '-o', color='lightskyblue',
             markeredgecolor="teal", markersize=3, markeredgewidth=1, label = 'Train')
    plt.plot(x_ticks, record['val_'+name], '-o', color='pink',
             markeredgecolor="salmon", markersize=3, markeredgewidth=1, label = 'Val')
    plt.grid(color='lightgray', linestyle='--', linewidth=1)

    plt.title('Model', fontsize=14)
    plt.ylabel(name, fontsize=12)
    plt.xlabel('Epoch', fontsize=12)
    plt.xticks(x_ticks, fontsize=12)
    plt.yticks(fontsize=12)
    plt.legend(loc='lower right' if not name.lower().endswith('loss') else 'upper right')

    # define saved figure or not
    if img_save:
        plt.savefig(f'result_img/{name}.png', transparent=False, dpi=300)
    if show:
        plt.show()

    plt.close()

In [23]:
history = train(model, train_loader, val_loader, optimizer, parameters, device)

# draw all metrics figure
draw_pic(history, 'loss', img_save=True, show=False)
draw_pic(history, 'acc', img_save=True, show=False)
draw_pic(history, 'f1', img_save=True, show=False)
draw_pic(history, 'rec', img_save=True, show=False)
draw_pic(history, 'prec', img_save=True, show=False)

files = []
files.append('result_img/loss.png')
files.append('result_img/acc.png')
files.append('result_img/f1.png')
files.append('result_img/rec.png')
files.append('result_img/prec.png')

[epoch 1] cost time: 32.8882 s
         loss     acc     f1      rec    prec
train | 0.6806, 0.8519, 0.8642, 0.8519, 0.8950
val  | 0.4765, 0.9342, 0.9426, 0.9342, 0.9567

Model saved to ==> ./model/10-28-19-03_epoch0.pt
[epoch 2] cost time: 32.6644 s
         loss     acc     f1      rec    prec
train | 0.5284, 0.9311, 0.9313, 0.9311, 0.9381
val  | 0.5421, 0.8962, 0.8934, 0.8962, 0.9093

Model saved to ==> ./model/10-28-19-03_epoch1.pt
[epoch 3] cost time: 32.5768 s
         loss     acc     f1      rec    prec
train | 0.5153, 0.9327, 0.9319, 0.9327, 0.9394
val  | 0.5461, 0.8733, 0.8647, 0.8733, 0.8942

Model saved to ==> ./model/10-28-19-03_epoch2.pt


In [24]:
def Softmax(x):
    return torch.exp(x) / torch.exp(x).sum()

# predict a single sentence
def predict_one(query, model):

  tokenizer = AutoTokenizer.from_pretrained(parameters['config'])
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  model.eval()
  with torch.no_grad():
    inputs = tokenizer.encode_plus(
            query,
            max_length = parameters['max_len'],
            truncation = True,
            padding = 'max_length',
            return_tensors = 'pt'
        )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    token_type_ids = inputs["token_type_ids"].to(device)

    # forward pass
    logits = model(input_ids, attention_mask, token_type_ids)
    probs = Softmax(logits) # get each class-probs
    label_index = torch.argmax(probs[0], dim=0)
    pred = label_index.item()

  return probs, pred

In [28]:
# You can load the model from the existing result
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
init_model = BertClassifier.from_pretrained(parameters['config'], parameters) # build an initial model
model = load_checkpoint('./model/10-28-19-03_epoch0.pt', init_model, device).to(device) # and load the weight of model from specify file

Some weights of BertClassifier were not initialized from the model checkpoint at nbroad/ESG-BERT and are newly initialized: ['classifier.hidden_layer.bias', 'classifier.hidden_layer.weight', 'dense.hidden_layer.bias', 'dense.hidden_layer.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\simsIab\AppData\Local\Temp\ipykernel_10236\1987939282.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode un


Model loaded from <== ./model/10-28-19-03_epoch0.pt


In [29]:
%%time
test = "Completely redesigned forn the first time since its debut 2007 model year, the 2016 Lincoln MKX is equipped with a number of advanced safety and driver assist technologies."
probs, pred = predict_one(test, model)
print(probs, pred)

# '''
# tensor([[0.9779, 0.0221]], device='cuda:0') 0
# CPU times: user 78.1 ms, sys: 4 ms, total: 82.1 ms
# Wall time: 340 ms
# '''

tensor([[0.1132, 0.1009, 0.1001, 0.6859]], device='cuda:0') 3
CPU times: total: 172 ms
Wall time: 353 ms


In [30]:
# 資料測試
test_df = pd.read_csv('./data/test.tsv', sep = '\t').reset_index(drop=True)
test_dataset = CustomDataset('val', val_df, 'text', parameters)
test_loader = DataLoader(test_dataset, batch_size=parameters['batch_size'], shuffle=True)
test_loss, test_acc, test_f1, test_rec, test_prec = evaluate(model, test_loader, device)
print('         loss     acc     f1      rec    prec')
print('test  | %.4f, %.4f, %.4f, %.4f, %.4f\n'%(test_loss, test_acc, test_f1, test_rec, test_prec))

         loss     acc     f1      rec    prec
test  | 0.4760, 0.9346, 0.9425, 0.9346, 0.9567

